In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import os

# 1. Codeup Blog Articles

Visit Codeup's Blog and record the urls for at least 5 distinct blog posts. For each post, you should scrape at least the post's title and content.

Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article. The shape of each dictionary should look like this:


{
    'title': 'the title of the article',
    'content': 'the full text content of the article'
}
Plus any additional properties you think might be helpful.

**Bonus**: Scrape the text of all the articles linked on codeup's blog page.

In [2]:
url = 'https://codeup.com/blog/'
headers = {'User-Agent': 'Codeup Data Science'}
response = get(url, headers=headers)

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
def get_blog_links(soup, existing_list):
    blog_list = existing_list
    for a in soup.select('a'):
        if 'read more' in a.text:
            blog_list.append(a['href'])
    return blog_list

In [5]:
def get_next_page(soup):
    for a in soup.select('a'):
        if 'Older Entries' in a.text:
            return a['href']

In [6]:
def get_soup(url):
    headers = {'User-Agent': 'Codeup Data Science'}
    response =  get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [7]:
soup = get_soup(url)
existing_list = []
links = get_blog_links(soup, existing_list)
links

['https://codeup.com/workshops/in-person-workshop-learn-to-code-python-on-7-19/',
 'https://codeup.com/workshops/dallas/free-javascript-workshop-at-codeup-dallas-on-6-28/',
 'https://codeup.com/tips-for-prospective-students/is-our-cloud-administration-program-right-for-you/',
 'https://codeup.com/workshops/pride-in-tech-panel/',
 'https://codeup.com/codeup-news/inclusion-at-codeup-during-pride-month-and-always/',
 'https://codeup.com/tips-for-prospective-students/mental-health-first-aid-training/',
 'https://codeup.com/workshops/codeup-dallas-how-to-succeed-at-a-coding-bootcamp-on-june-9th/',
 'https://codeup.com/featured/5-reasons-to-attend-our-new-cloud-administration-program/',
 'https://codeup.com/workshops/learn-to-code-python-on-5-21/',
 'https://codeup.com/dallas-newsletter/codeup-dallas-joins-career-day/',
 'https://codeup.com/workshops/project-quest-info-session-may-24th/',
 'https://codeup.com/workshops/from-bootcamp-to-bootcamp-a-military-appreciation-panel/',
 'https://code

In [8]:
url = get_next_page(soup)
url

'https://codeup.com/blog/page/2/?et_blog'

In [9]:
def get_all_blog_links(initial_url):
    existing_list = []
    url = initial_url
    while True:
        soup = get_soup(url)
        existing_list = get_blog_links(soup, existing_list)
        url = get_next_page(soup)
        if url == None:
            return existing_list

In [10]:
master_list = get_all_blog_links('https://codeup.com/blog/')

In [11]:
test = get_soup(master_list[0])

In [12]:
test.title.text

'In-Person Workshop: Learn to Code - Python on 7/19 - Codeup'

In [13]:
def get_all_info():
    master_list = get_all_blog_links('https://codeup.com/blog/')
    title = []
    content = []
    link = []
    date = []
    for i in range(len(master_list)):
        page = get_soup(master_list[i])
        date.append(page('p')[0].text.split('|')[0])
        title.append(page.title.text)
        link.append(master_list[i])
        content.append(page.article.text.strip())
    all_info = {'title':title, 'date':date, 'content':content, 'link':link}
    df = pd.DataFrame(all_info)
    return df

In [14]:
df = get_all_info()

KeyboardInterrupt: 

In [ ]:
df.to_csv('master_blog_list.csv', index=False)

In [ ]:
df.head()

In [ ]:
os.path.isfile('master_blog_list.csv') == False

# 2. News Articles

We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

Business
Sports
Technology
Entertainment
The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:


{
    'title': 'The article title',
    'content': 'The article content',
    'category': 'business' # for example
}
Hints:

Start by inspecting the website in your browser. Figure out which elements will be useful.
Start by creating a function that handles a single article and produces a dictionary like the one above.
Next create a function that will find all the articles on a single page and call the function you created in the last step for every article on the page.
Now create a function that will use the previous two functions to scrape the articles from all the pages that you need, and do any additional processing that needs to be done.


In [ ]:
base_url = 'https://inshorts.com/en/read/'
category_list = ['business', 'sports', 'technology', 'entertainment']

In [ ]:
url = base_url + category_list[0]
headers = {'User-Agent': 'Codeup Data Science'}
response = get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup.select(".news-card")

In [ ]:
business = soup.select('div[class = "news-card z-depth-1"]')
for i in range(len(business)):
    print(business[i].select('span[itemprop = "headline"]')[0].text)
    print(business[i].select('div[itemprop="articleBody"]')[0].text)

In [ ]:
business

In [ ]:
def get_all_articles():
    # Setting base variables
    base_url = 'https://inshorts.com/en/read/'
    category_list = ['business', 'sports', 'technology', 'entertainment']
    full_list = []
    # Making a for loop to go to the pages of all the categories
    for cat in category_list:
        # Getting the soup object
        url = base_url + cat
        headers = {'User-Agent': 'Codeup Data Science'}
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Getting the page articles
        page = soup.select('div[class = "news-card z-depth-1"]')
        # Getting the article title and content for the page
        for i in range(len(page)):
            single = {'title': (page[i].select('span[itemprop = "headline"]')[0].text), 
                      'content': (page[i].select('div[itemprop="articleBody"]')[0].text) , 
                      'category': cat}
            full_list.append(single)
    return full_list

In [ ]:
final = get_all_articles()

In [ ]:
df = pd.DataFrame(final)

In [ ]:
df.to_csv('in_shorts_articles.csv', index=False)

In [ ]:
df